In [2]:
from pytube import Playlist
from youtube_transcript_api import YouTubeTranscriptApi 

In [3]:
NDTV_Profit = Playlist("https://www.youtube.com/playlist?list=PLHQtHjXocftcYYP0b3fd8gUFwCGZsE6C4")
NDTV_Profit
type(NDTV_Profit)

pytube.contrib.playlist.Playlist

In [25]:
from datetime import datetime, timedelta
from pytube import Playlist, YouTube
def filter_videos_by_date(playlist, days_old=2):
    recent_videos = []
    one_day_ago = datetime.now() - timedelta(days=days_old)
    
    for video_url in playlist:
        try:
            # Create YouTube object for each video
            yt = YouTube(video_url)
            # Check the publish date
            if yt.publish_date >= one_day_ago:
                recent_videos.append(video_url)
        except Exception as e:
            print(f"Error processing video {video_url}: {e}")
    
    return recent_videos
    

In [26]:
NDTV_Transcript = []
recent_videos = filter_videos_by_date(NDTV_Profit)
for video_url in recent_videos:
    video_id = video_url.split("v=")[1]
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        NDTV_Transcript.append(transcript)
    except:
        print(f"No transcript found for video ID {video_id}")


In [27]:
NDTV_Transcript =[ " ".join(segment['text'] for segment in srt) for srt in NDTV_Transcript]


In [18]:
NDTV_Transcript[0:2]

["abishek kapor group CEO purvankara limited uh you know we'll continue our chat uh abishek you know I want to quickly first off understand from you uh firstly welcome good morning uh good morning let me understand from you how demand is playing out currently because uh the last we spoke you spoke about very robust demand you spoke about execution being uh something that youall are looking at very actively uh and that's basically the focus has shifted towards execution given the kind of demand that there is uh is demand really holding up and how is pricing shipping up for you hsh uh as an industry I can share with you I think demand is continuing to hold up uh the challenge really is on the supply side as I mentioned earlier uh to bring in new launches and the supply is is still continues to be a challenge in most of the markets that we operate in um I would expect expect now with the festive season that the demand will be better than the first quarter and then going forward the third 

In [28]:
len(NDTV_Transcript)

6

In [29]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

# Load environment variables from .env file
load_dotenv()

# Correctly fetch the API key using os.getenv()
api_key = os.getenv("GEMINI_API_KEY")

# Ensure the API key exists
if not api_key:
    raise ValueError("GEMINI_API_KEY not found in environment variables")

# Configure the API key using the fetched API key
genai.configure(api_key=api_key)

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "application/json",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-pro",
  generation_config=generation_config,
  # safety_settings = Adjust safety settings
  # See https://ai.google.dev/gemini-api/docs/safety-settings
  system_instruction="Make the information short and keep the important things only",
)

chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        "classify the text according to stock names\n[\"abishek kapor group CEO purvankara limited uh you know we'll continue our chat uh abishek you know I want to quickly first off understand from you uh firstly welcome good morning uh good morning let me understand from you how demand is playing out currently because uh the last we spoke you spoke about very robust demand you spoke about execution being uh something that youall are looking at very actively uh and that's basically the focus has shifted towards execution given the kind of demand that there is uh is demand really holding up and how is pricing shipping up for you hsh uh as an industry I can share with you I think demand is continuing to hold up uh the challenge really is on the supply side as I mentioned earlier uh to bring in new launches and the supply is is still continues to be a challenge in most of the markets that we operate in um I would expect expect now with the festive season that the demand will be better than the first quarter and then going forward the third and the fourth quarter definitely will be a a better quarter as it is you know annually because that's the general real estate cycle especially in residential for us uh most of the demand uh really comes in the third and fourth quarter so I think the industry is looking forward to the festive season right um abishek mahima also joining in um you know um you have a couple of flat a couple of flats in pipeline I want to understand what is happening on the execution side um of things and overall in fi25 what kind of um overall execution timeline are you looking at Mya thank you um so you know as far as executions concerned last quarter we delivered about a million plus square foot our goal this year is to deliver over a 4 million square foot and that's a Target we are working towards um and uh we believe that as we increase our launches uh for example last year we sold more than 7 million square foot uh this year we are launching about 17 million square foot of which about 12 million square foot is uh new launches and 4 million plus is uh from the previous fa previous uh phases of previous launches so uh we expect that the delivery will continue to pick up in the coming years uh and that continues to be our focus and of course that is resulting into the cash flows and collections because if you see now our toally run rate of collections has gone over a, crores uh and we will continue to see that Trend improving got it and how is the execution pipeline looking is Q2 Q3 Q4 pretty much balanced in terms of execution or do you believe that a lot of it will be back ended to a second half of this year look U the first quarter as I said over a million square foot uh we expecting the second quarter to be pretty much similar in nature uh a lot of delivery will happen happen in the fourth quarter for sure uh third quarter uh we will see where we land because it's also a question of uh you know how many um occupation certificates we are able to get upon completion because we have to go through platora of uh uh you know approvals in terms of completion before we start the handing over process and from the date we get the OC there is a process also of handing over um while I say that I must tell you that uh you know we have about 32 million sare foot under production and we continue to focus on that entire 32 million sare foot and on an ongoing basis if you look at our burn rate um you know our uh Quantum of money spent on operations that is continuing to scale up which basically means that eventually this uh product which is currently being constructed and the money being spent on will come to delivery all right and um abishek in q1 we saw that Providence segment had a significant increase of bookings but the L luxury segment seees some did see some kind of decline so uh what is the kind of strategy that you are adopting uh which is the kind of segment that you're focusing on one thing and what are the kind of Trends overall that you're you know observing in the market see you know uh our uh demand as we see it is pretty much homogeneous across all categories whether it is mid mass or luxury category uh in fact in the first couple of quarters we have had robus performance uh from Provident also on the back of the new launches that we' have had in Provident uh we expecting the launches to come through in purvankara uh in the coming quarters as I said the challenge really is more on the launch side and the supply side and as those launches kick in you will see a fairly balanced uh growth in both the numbers uh including Pura land you know we continue to focus on that product as well uh so you know uh we have an annual goal I think we have shared the fact that we are going to launch about 17 million square foot our goal will be to make sure that we are on target there and u what's the target with regard to overall uh collections for FY 25 how are we shaping up there uh like I mentioned you know our annual our quarterly run rate is about 1,000 so you know we are at least at 4,000 for sure but you add to that uh new launches I'm sure you will see much better collections we intend to continue to grow the collections of course operations and delivery will also help uh so we will continue to see upward momentum in the collection especially in the third and the fourth quarter as we do the new launches right and what about realizations so you're talking about such robust demand continuing uh do you therefore is is there a continued uptick where realizations is concerned and how much of an optic really because the base is getting higher now look I think you know as an industry uh we've been very consistent in the price increase now specifically in last couple of quarters uh my sense is uh that we would we would be uh expecting a price increase of you know single digits uh more inflation link I would expect it to be average industry uh escalation of 8 to 9% annually but from Project to project basis you will see different price increases in different micro markets I think that's where the that's where the Nuance is but as far as puran and Provident in Pura land is concerned uh you know again uh it's more a factor of mix of inventory that we are bringing to the market and mix of inventory that is getting sold and therefore you will see while on an individual project level you will see continued appreciation but on an overall realization level you will see depending on the mix of inventory that we bring to the market on on how the average realization moves for example you know if you look at purva land typically our realizations are any anywhere between 3,000 Rupees to 5,000 rupees a square foot whereas in Provident it would be somewhere around 8,000 and purva will be 10,000 plus average ization so depending on the mix and volumes that you sell you will see the uh overall average moving uh however on a project level basis you will continue to see the realization gr all right and U in terms of debt you know your net debt has uh seen a slight uptake in q1 um as compared to Q4 um I I want to understand that what are your plans with respect to managing these debt levels and where will the debt level be at by the end of fi25 look as I mentioned uh you know our debt we are expecting to keep it at similar levels our goal is on a per square foot basis to be uh uh to keep the debt under 1,000 rupes a square foot of area under development and we'll continue to Target that number uh and on an absolute number maintain it at similar levels uh it may marginally go up and then may come down depending on you know the transactions that we are doing because it's a constant process of churn of debt uh because debt is getting constantly repaid as well as we are doing also new acquis positions so you know it will be a consistent churn but we will our Target is to keep it at similar Levels by the end of the year okay and with regard to geographies of course new launches you've spoken about but what's talk to us about the next three years right what's the kind of thought process you're looking at uh looking at is it going to be more focused on Bangalore uh Kochi Chennai and the like or I going to move towards a Pune or a Mumbai and Beyond look that's pretty much reflective of what's Happening currently in the acquisition I mean if you see our last uh this fin this calendar year Acquisitions uh you know we have done acquisition in tan in Mumbai uh we have signed up in loala in fact we've just added four more Societies in loala we have just announced a project in South Mumbai our first project in breach candy um we have also been appointed as a preferred developer in Pali Hill uh if you look at uh you know the Quant of Investments that's happening in the west clearly indicates that in future we will have significant contribution coming in from the West while we continue to consolidate and announce our Acquisitions in in South for example uh we have acquired two projects in um uh in electronic City area in Bangalore and I'm certain that you will see more and more announcements as far as the southern markets are concerned so our goal is going to be definitely to make sure in South we consolidate our position and in the west we expand our position uh while we have now started to explore NCR and we are hoping that in next two years to three years time frame we'll start expecting some contribution coming in from the NCR region all right well abishek thank you so much for giving us those insights on purvankara and thank you so much for taking our time and speaking with us at NV profit uh but with that we're completely out of time on this edition of small midcap show but keep watching NV profit for more news and updates [Music]\",\n \"hello and welcome you're watching the small and midcap show I'm mahima VRA Jani and with me is H sa well um we have two very interesting managements lined up for you today the first one is e my trip where lots happening with this company and we are joined by Mr Prashant py who's the co-founder at e my trips who joins us now uh welcome to the show Prashant uh Prashant a lot is happening when it comes to um you know is my trip lot of multiple Acquisitions um so you know all over we we want to know uh you know in terms of what's happening what are the kind of Acquisitions that you're doing I'll start with uh you know the uh recent U investment in Rollins and fled which is you know into medical tourism so I want to understand that um you know what are the kind of synergies that you're trying to build in this particular acquisition now firstly happy to be on the show and provide explanation for the things which are happening at eart trip firstly before I would like to set the context uh the company has been growing ever since uh we are still a boot we were a bootst company till the time we got listed and from the listing till today our business has grown by almost four times the travel the cor travel business has grown by four times the profitability has almost grown by about five times uh over the period of last three years so the company is in very well solid position to continue and grow the travel related businesses uh however at the board level we have also recently decided that we are going to be bit Diversified and the idea behind that is that um you know as you would know that there are very few companies who have been extremely disciplined and and have been bootstrapped internet companies to become a boot stab listed company is a very very rare phenomena and from that you should understand that the promoters probably uh you know possess certain knowledge uh which are not very common and for that the board has decided that we will basically find another level of uh find another kind of companies like e my trip in different domains who are bootstrapped uh but would potentially be a listed company in the next three to four years time and we would be basically funding them and helping them with our advisory with our technology with our consumer base and help them grow faster than before and for this we are taking minority Stak in those companies and couple of Acquisitions which you have seen in the recent uh in respect to that right Prashant harsh also joining in uh is this is this collaboration of sorts only India Centric or or is it with regard to more geographies outside of India as well because medical tourism uh of course uh is is a growing sector in India it's also a niche so want to try and understand as to how this play is going to work for you so uh with respect to medical tourism it cannot be just India specific for example the pedg companies actually a Dubai based company uh they they already 30% of their business right now is actually allowing people whom they have identified have problems and can be solved in India related to surgeries they're actually you know sending those passengers to India they have tie-ups with various hospitals and they can directly contribute to eat business ISAT Dubai business eat Dubai business is growing rapidly last what we did business of about 126 crores and we believe P can be can help East M Dubai business monumentally uh by contributing to it so by the nature of medical tourism itself which is growing really rapidly and India become India is becoming a very hot destination for medical tourism uh we believe that it has to be at the global scale all right and Prashant what are the kind of revenues that you're expecting from this particular um acquisition and secondly um the margins in the uh medical tourism business um are much higher so are you expecting your Blended margins to go up absolutely uh and more than that but however these are minor minority acquisition so they will not directly uh play a role of you know onto ISAT trips margin since they are on the minority side the high level understanding is that in the future as well we might uh do a do a playoff like in 4 and grow the company uh in various other domains as well probably looking for more asset like technology driven companies which are something like ISAT bootstrap and for this we are going to very shortly uh start a website as well to allow people to apply uh their companies to become uh part you know so that e can invest in them so this is the larger player which we are looking at eeve is a whole different uh game you know we already have a company called as YOLO bus and YOLO bus is operating about 80 odd buses across India right now when we acquired the company it only had two three buses now it is about at 80 bus uh we really look forward for YOLO bus to grow to become 2,000 plus buses in the next couple of years and for that uh we we found out that the best unit economics is actually in the EV buses space not the diesel buses and with respect to that we started searching for Ev buses but we couldn't find many suppliers there's a huge shortage over there and that is when we decided that we will basically start a manufacturing of EV buses ourselves uh to to sport YOLO bus the operating arm to consume and also for uh other uh you know government agencies or other agencies to buy from us got it Prashant Prashant how quickly are you able to set this business up how quickly does the ball start to roll ESP especially on the EV bus side of things uh and with regard to yolobus so yolobus is already an operational company doing wonderfully well for us uh in terms of uh the the EV manufacturing unit we have identified the right person to become the CEO Mr Mano Sony who's a veteran in the industry comes from asok leand uh we have appointed him as a CEO of the company and uh he's rolling the ball right now we are in the phase of doing the R&D and figuring out the right product and then the next phase will come is basically to set up the manufacturing unit uh I believe in the matter of 2 years we should be able to launch the manufacturing unit and which which would have the capacity of rolling out 2,000 to 3,000 buses on annual basis and all of those buses will be used by you by EAS my trip or would you also be selling them outside maybe uh uh to government Etc correct so not necessarily all of them will be used internally by YOLO buses uh yes first demands for YOLO buses will be fulfilled and any extra capacity there's a huge short shortage of EV buses there is very high demand from the government sector sure roll out in EV buses for intra City and Inter City travel and uh we would look forward to fulfill those demands as well and that will shrink your margins that that part of the business not not really uh that's also as I said that there's a huge supply uh crunch in the market and hence uh the demand could be uh demand could could pay pretty well what will your margins on that particular business be what are you anticipating uh that number to be around around 10 to 15% while our current margins are at about 8% got it got it uh with regard to your incubation business now that's that's something which is differentiated because you spoke about infoed and how infoedge has done uh that's going to be unique so how much in terms of cash flow do you all do uh free cash flow do you all do annually and how much of that will you be putting to use uh for your incubation business and how much for your uh current business of e my trip which you are already venturing into several businesses R so as you would know e my trip is a you know free cash flowing company you know where we have a positive cash flow last year we did a cash flow of about 130 124 cores uh so we will as a profitable company we will utilize our free cash flow uh to basically invest in these uh young companies which which potentially can become a listed company very shortly we are also launching a platform called as ra once.com raise once basically means that you know just raise once to to be able to get to a listed space uh from bootstrap to listed just by raising once and hopefully that raise once will come from uh e my trip and U we're looking forward to help those companies to grow faster and get to a listing space all right uh so Prashant um you talked about this raise one thing so I want to understand more on that as well as you know you've also introduced scan my trip so I want to understand both these businesses first and then you know get into the financial aspect of that sure uh scan mrip is basically in collaboration with WC this is first of its own kind where eatrip has become you know a seller and as well as a buyer of travel related Services uh we have Co jointly created the apis and uh on on the basis of this you know we will be able to sell and buy on onc onc is picking up really well and we want to be ahead of the curve which is why is matter it's the first OTA which is l launch this scan my trip uh which will allow various lacks and lacks of buyers and suppliers on WC to utilize our services all right so this is targeted more towards uh companies per se companies and corporates not necessarily there are a lot of msmes which are already using onc so for for msme bookings for individual bookings uh onc is a platform is it's a protocol basically uh an exchange of an API and uh scan mrip has basically become one of the buyers and the suppliers uh both uh on onc okay and then what okay and then what is the kind of revenues that you're expecting from this particular um I mean uh scan my trip introduction and uh what are the kind of margins there so again the uh you know it's too early for us to say all this uh because you know we are the Pioneers in the industry this is the first OTA which is getting launched on onc platform enhance it would be difficult for me to to predict what kind of volumes we should expect however we would know we we already know that onc is highly highly successful uh in basically getting the adoption and if the adoption has come for non-travel uh usage I'm assuming that the adoption will also come for travel usage and hence I'm looking forward to see numbers grow rapidly onc and will that affect your business Prashant because isn't that in some form competition for your business not really uh because besides the flight booking there are ious other bookings for which e does not have very high market share for example in the holiday space for example in the you know transfer space ISAT does not have a very high margin market share and uh onc I believe will actually add a lot of value in such spaces got it and Prashant you know with regard to Quantum uh is there a particular ballpark number that you all will be looking to invest uh through your new platform which you are putting up uh into companies uh raise once as you call it how much would that Quantum there is no Quantum which we have set right now both the deals which have happened are actually Equity swap at the moment the ones which is with the rolin and the P fledge okay and hence there is no cash outf flow which is happening for this 90 deal okay so Rollin is part of these Acquisitions which are going to be making these minority Acquisitions correct rolin and P fles both are part of this minority acquisition which we are making and uh I'm looking forward to uh more such companies which are basically doing wonderfully well uh and could become the next ease Market got it and how is the medical tourism uh vertical going sorry I'm coming probably back to where we started but how is that one going so how does that start contributing both to uh Top Line uh are you doing it Standalone by yourselves are you all doing it in partnership with someone and what are the Target geographies that you have so medical tourism is actually a very there's a lot of there are a lot of nuances around it it's it's a beautiful industry rapidly growing India is becoming a hot destination for medical tourism and which is why we have forid into it along with the pedg pedg is one of the leading service providers for people in Middle East and CIS countries and help them to come to India for their medical needs and uh we expect that this business is going to ramp up pretty rapidly and you know we look forward to become part of it as a shareholder and as well as travel service providers to the people who are coming to India for medical tourism uh do it only through P fled is that how one looks at it Mr py uh for the countries which are in Middle East and CIS yes we would do it through ped because they need to be the originator of customers the customers and we we are not we don't know those customers pedg what it does is it provides home care to people in Middle East now when they providing home care to people in mle E they are already in touch with customers and hence they know people who need medical tourism to come to India to get their ailment right and hence they become the source of the origin of the customers who are coming to India and hence we would work along with them and do you all get a revenue share as part of this of course 30% is your ownership in that company but do you all also get a revenue share out of this does it contribute to Top Line absolutely and what is that percentage if at all uh that is not something public in future we probably will uh disclose more as of these geographies would you also be exploring us absolutely okay absolutely so wherever there is Big medical tourism we are looking for companies like page based out of those company those countries who can basically do origination for medical tourism to create demand for India and have you all tied up with Partners in India uh correspondingly because that's something you all will need so pedge already has multiple hospitals with whom they working along with our responsibility will be to take care of the travel part understood that makes sense got it uh and with regard to the 1,000 CR fund raise uh of course you've also suggested 200 CR to be invested in your manufacturing facility uh just one quick clarification so where does that 1,000 CR go or where does the balance 800 CR go and is the capacity going to be at 4 to 5,000 buses or 2 to 3,000 buses the capacity annual capacity is going to be 2 to 3,000 uh basis and uh the fund raise uh we have taken a board approval to raise up to 1,000 CR rupees we are speaking to few investors uh to see if we can uh you know if there is a demand right now there is uh basically we we are in the phase of understanding the demand uh and but uh the board approval is already there and as and when any next steps happen we will be informing you guys okay thank you got it thank you so much Prashant has been a pleasure chatting of course uh lots happening with e my trip wish you all the very best and uh you know we'll chat with you very very soon of course post the q1 numbers or Q2 numbers I beg your pardon uh to get better stock of what's Happening thanks so much thank you thank you [Music]\"]\n",
      ],
    },
    {
      "role": "model",
      "parts": [
        "```json\n[\n  {\n    \"stock_name\": \"Purvankara Limited\",\n    \"summary\": \"Purvankara Limited's Q1 earnings call discussed strong housing demand, especially in Q3 and Q4, driven by the festive season. They are focused on execution and increasing supply to meet this demand. New launches are expected to drive growth, with a target of 17 million sq ft for FY25.  They aim to maintain debt levels and see continued price appreciation. Geographically, they are consolidating in South India and expanding in the West, with plans to enter the National Capital Region (NCR) market in the next 2-3 years.\"\n  },\n  {\n    \"stock_name\": \"EaseMyTrip\",\n    \"summary\": \"EaseMyTrip's recent strategic moves include minority investments in medical tourism (Rollins & Pledg), expanding their EV bus fleet (YOLO Bus), and establishing a manufacturing unit for EV buses. They plan to invest in promising startups with the potential to go public (RaiseOne.com platform). Other initiatives include 'ScanMyTrip' in partnership with ONDC to tap into corporate travel. They aim to raise up to ₹1000 crore, with ₹200 crore allocated to the EV bus manufacturing facility.\"\n  }\n]\n```",
      ],
    },
  ]
)

response = chat_session.send_message(NDTV_Transcript)

print(response.text)

[
{
"stock_name": "Purvankara Limited",
"summary": "Purvankara Limited expects strong housing demand, driven by the festive season in Q3 and Q4. They are focused on execution and increasing supply to meet this demand. New launches are expected to drive growth. Geographically, they are consolidating in South India and expanding in the West, with plans to enter the National Capital Region (NCR) market in the next 2-3 years."
},
{
"stock_name": "EaseMyTrip",
"summary": "EaseMyTrip is making minority investments in medical tourism and expanding into the EV bus segment with a new manufacturing unit. They plan to invest in promising startups through their RaiseOne.com platform. Other initiatives include 'ScanMyTrip' in partnership with ONDC. They aim to raise up to ₹1000 crore."
},
{
"stock_name": "JSW Steel",
"summary": "JSW Steel is expected to benefit from its domestic focus, as India's steel consumption remains strong. The company is focusing on value-added products to mitigate the impac